In [2]:
from google.colab import drive ; drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install transformers
!pip install pytypo
!pip install emoji

     |████████████████████████████████| 1.8MB 7.4MB/s 
     |████████████████████████████████| 2.9MB 34.7MB/s 
     |████████████████████████████████| 890kB 47.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=036effc0e9d4417adf6d5331918b20944867d2c92064b9ac36678b16d6d8e456
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 81kB 5.1MB/s 
  Created wheel for pytypo: filename=pytypo-0.3.0-cp36-none-any.whl size=72688 sha256=ca3b002398e892efb4e6e27733f57eca8d06efd19c9c014d7554c742edf0a360
  Stored in directory: /root/.cache/pip/wheels/f3/81/55/d305159b9e1631c244110dc3b131b1d0dcf041672dd2e8de9a
Successfully built pytypo
     |████████████████████████████████| 133kB 7.5MB/s 


In [4]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import sklearn as skl
import pytypo
from re import sub,compile,UNICODE
import emoji
pd.set_option("max_rows",None)

AUTO = tf.data.experimental.AUTOTUNE

In [5]:
dfTrain = pd.read_csv("/content/drive/My Drive/Dataset/penyisihan-datavidia-7-0/train.csv",sep=",")
dfTest = pd.read_csv("/content/drive/My Drive/Dataset/penyisihan-datavidia-7-0/test.csv",sep=",")

In [6]:
dfTrain.head()

,review_id,review_text,category
0,0fc651bfdd38a3b4ba19ef9d0d1e92e3,"Penginapannya rapi, bersih, suasana tenang, pe...",1
1,7d8cb2e465c5602e51c1ba8710e8ff16,"wifi tidak sampai kamar, tidak tersedia heater...",0
2,1f4447c73e5e1aac462d05b0ecbd5a55,"Lantai kamar mandi kotor, bau, dan seperti sdh...",0
3,b9aefbfb0debbfff43f8b268c82b43c0,"Wc jorok.. Kasur tidak dibersihkan,, handuk ti...",0
4,e2afef18bd695fae43d4d53b8c547bd4,bocor lagi,0


In [7]:
print(f"Row {dfTrain.shape[0]} \n Col {dfTrain.shape[1]}") # row = 14856, col = 3
print(f"Col Names {dfTrain.columns}") 

Row 14856 
 Col 3
Col Names Index(['review_id', 'review_text', 'category'], dtype='object')


In [8]:
dfTrain = dfTrain.drop(["review_id"] , axis=1) # Dropped
dfTrain.head()

,review_text,category
0,"Penginapannya rapi, bersih, suasana tenang, pe...",1
1,"wifi tidak sampai kamar, tidak tersedia heater...",0
2,"Lantai kamar mandi kotor, bau, dan seperti sdh...",0
3,"Wc jorok.. Kasur tidak dibersihkan,, handuk ti...",0
4,bocor lagi,0


In [9]:
print(f"How Many Nulls  ? {dfTrain['category'].isnull().sum()}") # 0
print(f"How Many Values For 1 and 0 \n{dfTrain['category'].value_counts()} ") # Imbalance (0 : 12839, 1 : 2017)


How Many Nulls  ? 0
How Many Values For 1 and 0 
0    12839
1     2017
Name: category, dtype: int64 


In [10]:
def clean_text(text):

  text_cleaning_re = "@\S+|https?:\S+|http?:\S|[#]+|[^A-Za-z0-9]+"
  text_cleaning_hash = "#[A-Za-z0-9]+" 
  text_cleaning_num = "(^|\W)\d+"

  
  text = sub(text_cleaning_hash, " ", text).strip()
  text = sub(text_cleaning_num, " ", text).strip()
  text = sub(text_cleaning_re, " ", text).strip()
  text = text.lower().strip()
  text = "".join(text.split("."))  
  out = []
  for word in text.split() :
    out.append(word)

  return pytypo.correct_sentence(" ".join(out).strip())

clean_text("cleaning Gan.. 213213902183 httP:////........ ")

'cleaning gan http'

In [18]:
for i in range(0, len(dfTrain) // 3):
  print(f" Loc : {i} {dfTrain.loc[i,'review_text'] }" )

 Loc : 0 penginapannya rapi bersih suasana tenang pelayannya ramah dan banyak membantu lokasi cukup dekat ke beberapa obyek wisata kalau saya ke bandung lagi mungkin saya akan menginap lagi di sini
 Loc : 1 wifi tidak sampai kamar tidak tersedia heater untuk kopi the atau dispenser dan tidak tersedia kursi di depan kamar untuk merokok hingga harus duduk di lantai
 Loc : 2 lantai kamar mandi kotor bau dan seperti sdh jdi sarang nyamuk dikamar mandinya pesan superior double yg dikasih twin mungkin karena murah saya maklumin tpi kedepan mohon untuk bisa jdi perbaikan untuk bisa lbh baik lg
 Loc : 3 wc jorok kasur tidak dibersihkan handuk tidak diganti
 Loc : 4 bocor lagi
 Loc : 5 kebersihan pelayanan kerapihan
 Loc : 6 saya hampir tidak bisa tidur krn bantal sangat bau apek
 Loc : 7 pelayanannya sangat baik kmr yang bersih menu makanannya juga enak to sayang wifinya kutang bagus
 Loc : 8 kamar banyak kecoa tlg lebih diperhatikan kebersihannya
 Loc : 9 kamar mandi banjir
 Loc : 10 lebih me

In [14]:
def clean_emoji(text):
    emoji_patterns = compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  
                               u"\U00002500-\U00002BEF"  
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  
                               u"\u3030"
                               "]+", flags=UNICODE)
    return emoji_patterns.sub(r'', text)
print(f"Before :{dfTrain.loc[14492  ,'review_text']}")
clean_emoji(dfTrain.loc[14492  ,"review_text"])

Before :kamarnya sempit toilet apalagi water heater y jg ga berfungsi trus nemuin kelabang di lantai kamar dan di atas kasur omg


'kamarnya sempit toilet apalagi water heater y jg ga berfungsi trus nemuin kelabang di lantai kamar dan di atas kasur omg'

In [16]:
dfTrain["review_text"] = dfTrain["review_text"].apply(clean_text)
dfTrain["review_text"] = dfTrain['review_text'].apply(clean_emoji)
print("Cleaned")

Cleaned


In [50]:
UniqueWord = set()
def Uniques(df):
  for i in range(len(df)):
    Listeddesu = (df.loc[i,"review_text"].split(" "))
    for j in Listeddesu:
      UniqueWord.add(j)
Uniques(dfTrain)

In [54]:
print(f"There are : {len(UniqueWord)} words" )

There are : 13071 words


In [60]:
ShortWord = []
MediumWord = []

def takeShortestword(UniqueWord):
  for i in UniqueWord:
    if len(i) < 5:
      ShortWord.append(i)
  for i in UniqueWord:
    if len(i) > 5 and len(i) < 8:
      MediumWord.append(i)
takeShortestword(UniqueWord)
len(ShortWord) , len(MediumWord)

(2417, 4047)

In [71]:
print(ShortWord[1:5], MediumWord[1:5]) 

['ice', 'lau', 'halo', 'bsd'] ['ribett', 'belagak', 'terbaik', 'ampela']


In [72]:
for i in range(len(ShortWord)):
  print(ShortWord[i])


ice
lau
halo
bsd
gdk
inc
hrga
far
gh
deng
ok
krg
must
msti
sudh
jm7
hour
klau
bus
bpkb
isn
sich
puja
arus
visa
hemm
hu
soto
grab
dong
tid
mie
mt
stu
rvti
aiy
m2
hny
cor
bnyk
gsm
gmbr
pas
egak
bank
be
dini
mrk
diri
tisu
jual
kasa
by
koko
yeah
kita
raib
rban
nga
csb
hut
via
hi
sdg
foil
jd
ya
atap
gue
btl
niat
only
tas
smua
wifi
yaah
odol
c7
wr
bagi
mei
tell
rias
enam
t
lenx
alas
dkmr
umur
lha
pdam
kaos
byar
mlah
here
car
deep
lagu
cai
utuh
sis
mati
loyo
dgn
pvj
kami
meja
tyt
ddlm
mkan
res
jaga
took
ber
mrcc
mhn
tk
roos
kuti
kaya
liar
peni
mw
fo
rp1
mbil
idk
same
okke
ato
dna
wah2
pipa
kem
mnd
brr
dkt
blg
many
cr
lmpu
c
shif
hari
mood
dr
seh
bag
boc
alfa
raik
nyaa
will
bbrp
cuek
tny
bgtu
lain
buru
mint
bego
giok
asin
bude
kmu
kurs
coyy
elek
skli
awal
ex
scam
aren
open
bulu
kue2
prlu
cair
maaf
urus
tele
rsk
daki
nmor
br
tuk
skat
enak
ckp
sek
salh
fake
lt
well
paid
nge
sepi
suru
jln
kos2
are
adem
ha
info
wait
exos
roof
mala
bath
bk
suka
best
mkin
pos
bisa
cafe
was
pe
bauu
eeh
cuti
baca
us
